In [12]:
def conj_set_preimage_no_independence(self, other):
    from itertools import product
    from copy import deepcopy
    f = copy(self)
    g = copy(other)
    n = f.domain().dimension_relative()
    if (n == 1) and (R in NumberFields() or R in FiniteFields())\
        and (f.sigma_invariants(1) != g.sigma_invariants(1)):
        return []
    
    def isLinearlyIndependent(S):
        M = matrix([list(t) for t in S])
        if len(S) < n+1:
            if M.rank() == len(S):
                return True
            return False
        else:
            if any(l == 0 for l in M.minors(n+1)):
                return False
            return True

    L = Set(f.periodic_points(1))
    K = Set(g.periodic_points(1))
    if len(L) != len(K):  # checks maps have the same number of fixed points
        return []
    mult_to_point_L = {}
    mult_to_point_K = {}
    point_to_mult_L = {}
    point_to_mult_K = {}
    level = 0
    for i in range(len(L)):
        mult_L = f.multiplier(L[i], 1).charpoly()
        mult_K = g.multiplier(K[i], 1).charpoly()
        tup_L = (mult_L, level)
        tup_K = (mult_K, level)
        if tup_L not in mult_to_point_L:
            mult_to_point_L[tup_L] = [L[i]]
        else:
            mult_to_point_L[tup_L] += [L[i]]
        if tup_K not in mult_to_point_K:
            mult_to_point_K[tup_K] = [K[i]]
        else:
            mult_to_point_K[tup_K] += [K[i]]
        point_to_mult_L[L[i]] = (mult_L, level)
        point_to_mult_K[K[i]] = (mult_K, level)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    repeated_mult_L = {}
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            return []
        elif len(mult_to_point_K[mult_L]) != repeated:
            return []
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]
    #print('repeated_mult:', repeated_mult_L)
    d = len(L)
    r = f.domain().base_ring()
    more = True
    source = []
    corresponding = []
    for r in sorted(repeated_mult_L.keys()):
        for point_lst in repeated_mult_L[r]:
            for point in point_lst:
                if isLinearlyIndependent(source + [point]):
                    source.append(point)
                    mult = point_to_mult_L[point]
                    # if another point with this multiplier and level pair is in S
                    # then the multiplier level pair will be the last element of corresponding
                    if len(corresponding) != 0:
                        if corresponding[-1][0] == mult:
                            corresponding[-1][1] += 1
                        else:
                            corresponding.append([mult, 1])
                    else:
                        corresponding.append([mult, 1])
                if len(source) == n+2:
                    more = False
                    break
            if len(source) == n+2:
                break
        if len(source) == n+2:
            break
    #print('corresponding:', corresponding)
    i_repeated_mult = deepcopy(repeated_mult_L)
    a_repeated_mult = {}
    found_no_more = True
    while more:
        level += 1
        for r in i_repeated_mult:
            for point_lst_L in i_repeated_mult[r]:
                old_tup_L = point_to_mult_L[point_lst_L[0]]
                #print('current tup:', old_tup_L)
                point_lst_K = mult_to_point_K[old_tup_L]
                mult_L = old_tup_L[0]
                Tl = []
                Tk = []
                for pnt in point_lst_L:
                    for preimage in f.rational_preimages(pnt):
                        if preimage != pnt:
                            Tl.append(preimage)
                for pnt in point_lst_K:
                    for preimage in g.rational_preimages(pnt):
                        if preimage != pnt:
                            Tk.append(preimage)
                #print('Tl:', Tl)
                #print('Tk:', Tk)
                if len(Tl) != len(Tk):
                    return []
                if len(Tl) != 0:
                    found_no_more = False
                    new_tup_L = (mult_L, level)
                    new_tup_K = (mult_L, level)
                    #print('new_tup_L:', new_tup_L)
                    mult_to_point_L[new_tup_L] = Tl
                    mult_to_point_K[new_tup_K] = Tk
                    for i in range(len(Tl)):
                        point_to_mult_L[Tl[i]] = new_tup_L
                        point_to_mult_K[Tk[i]] = new_tup_K
                    repeated = len(Tl)
                    if repeated not in repeated_mult_L:
                        repeated_mult_L[repeated] = [Tl]
                    else:
                        repeated_mult_L[repeated] += [Tl]
                    if repeated not in a_repeated_mult:
                        a_repeated_mult[repeated] = [Tl]
                    else:
                        a_repeated_mult[repeated] += [Tl]
                    # right now rechecking independence
                    source = []
                    corresponding = []
                    for r in sorted(repeated_mult_L.keys()):
                        for point_lst in repeated_mult_L[r]:
                            for point in point_lst:
                                if isLinearlyIndependent(source + [point]):
                                    source.append(point)
                                    mult = point_to_mult_L[point]
                                    # if another point with this multiplier and level pair is in S
                                    # then the multiplier level pair will be the last element of corresponding
                                    if len(corresponding) != 0:
                                        if corresponding[-1][0] == mult:
                                            corresponding[-1][1] += 1
                                        else:
                                            corresponding.append([mult, 1])
                                    else:
                                        corresponding.append([mult, 1])
                                if len(source) == n+2:
                                    more = False
                                    break
                            if len(source) == n+2:
                                break
                        if len(source) == n+2:
                            break
                if not more:
                    break
            if not more:
                break
        if found_no_more:
            raise ValueError('no more rational preimages')
        i_repeated_mult = deepcopy(a_repeated_mult)
        a_repeated_mult = {}
        found_no_more = True
        #print('i_repeated_mult:', i_repeated_mult)
    #print('source:', source)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    #print('repeated_mult_L keys:', sorted(repeated_mult_L.keys()))
    possible_targets = []
    for tup in corresponding:
        possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
    #print('possible_targets:', possible_targets)
    arrangements_iterators = []
    for lst in possible_targets:
        arrangements_iterators.append(Arrangements(range(len(lst[0])), lst[1]))
    #print('arrangement_iterators:', arrangements_iterators)
    #print('cardinalities:', [itr.cardinality() for itr in arrangements_iterators])
    count=0
    count2 = 0
    Conj = []
    for tup in product(*arrangements_iterators):
        #print('tup:', tup)
        current_target = []
        for i in range(len(tup)):
            for j in tup[i]:
                current_target.append(possible_targets[i][0][j])
        #print('current_target:', current_target)
        try:
            phi = f.domain().point_transformation_matrix(current_target, source)
            #print('phi:', phi)
            #print('phi inverse', phi.inverse())
            #print([phi*pnt for pnt in current_target])
            if self.conjugate(phi) == other:
                Conj.append(phi)
        except ValueError:
            count2 += 1
            pass
        count += 1
    #print('number of targets tried:', count)
    #print('number of not independent targets:', count2)
    return Conj

In [16]:
def conj_set_preimage_independece(self, other):
    from itertools import product
    from copy import deepcopy
    f = copy(self)
    g = copy(other)
    n = f.domain().dimension_relative()
    if (n == 1) and (R in NumberFields() or R in FiniteFields())\
        and (f.sigma_invariants(1) != g.sigma_invariants(1)):
        return []
    
    def isLinearlyIndependent(S):
        M = matrix([list(t) for t in S])
        if len(S) < n+1:
            if M.rank() == len(S):
                return True
            return False
        else:
            if any(l == 0 for l in M.minors(n+1)):
                return False
            return True

    L = Set(f.periodic_points(1))
    K = Set(g.periodic_points(1))
    if len(L) != len(K):  # checks maps have the same number of fixed points
        return []
    mult_to_point_L = {}
    mult_to_point_K = {}
    point_to_mult_L = {}
    point_to_mult_K = {}
    level = 0
    for i in range(len(L)):
        mult_L = f.multiplier(L[i], 1).charpoly()
        mult_K = g.multiplier(K[i], 1).charpoly()
        tup_L = (mult_L, level)
        tup_K = (mult_K, level)
        if tup_L not in mult_to_point_L:
            mult_to_point_L[tup_L] = [L[i]]
        else:
            mult_to_point_L[tup_L] += [L[i]]
        if tup_K not in mult_to_point_K:
            mult_to_point_K[tup_K] = [K[i]]
        else:
            mult_to_point_K[tup_K] += [K[i]]
        point_to_mult_L[L[i]] = (mult_L, level)
        point_to_mult_K[K[i]] = (mult_K, level)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    repeated_mult_L = {}
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            return []
        elif len(mult_to_point_K[mult_L]) != repeated:
            return []
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]
    #print('repeated_mult:', repeated_mult_L)
    d = len(L)
    r = f.domain().base_ring()
    more = True
    source = []
    corresponding = []
    for r in sorted(repeated_mult_L.keys()):
        for point_lst in repeated_mult_L[r]:
            for point in point_lst:
                if isLinearlyIndependent(source + [point]):
                    source.append(point)
                    mult = point_to_mult_L[point]
                    # if another point with this multiplier and level pair is in S
                    # then the multiplier level pair will be the last element of corresponding
                    if len(corresponding) != 0:
                        if corresponding[-1][0] == mult:
                            corresponding[-1][1] += 1
                        else:
                            corresponding.append([mult, 1])
                    else:
                        corresponding.append([mult, 1])
                if len(source) == n+2:
                    more = False
                    break
            if len(source) == n+2:
                break
        if len(source) == n+2:
            break
    #print('corresponding:', corresponding)
    i_repeated_mult = deepcopy(repeated_mult_L)
    a_repeated_mult = {}
    found_no_more = True
    while more:
        level += 1
        for r in i_repeated_mult:
            for point_lst_L in i_repeated_mult[r]:
                old_tup_L = point_to_mult_L[point_lst_L[0]]
                #print('current tup:', old_tup_L)
                point_lst_K = mult_to_point_K[old_tup_L]
                mult_L = old_tup_L[0]
                Tl = []
                Tk = []
                for pnt in point_lst_L:
                    for preimage in f.rational_preimages(pnt):
                        if preimage != pnt:
                            Tl.append(preimage)
                for pnt in point_lst_K:
                    for preimage in g.rational_preimages(pnt):
                        if preimage != pnt:
                            Tk.append(preimage)
                #print('Tl:', Tl)
                #print('Tk:', Tk)
                if len(Tl) != len(Tk):
                    return []
                if len(Tl) != 0:
                    found_no_more = False
                    new_tup_L = (mult_L, level)
                    new_tup_K = (mult_L, level)
                    #print('new_tup_L:', new_tup_L)
                    mult_to_point_L[new_tup_L] = Tl
                    mult_to_point_K[new_tup_K] = Tk
                    for i in range(len(Tl)):
                        point_to_mult_L[Tl[i]] = new_tup_L
                        point_to_mult_K[Tk[i]] = new_tup_K
                    repeated = len(Tl)
                    if repeated not in repeated_mult_L:
                        repeated_mult_L[repeated] = [Tl]
                    else:
                        repeated_mult_L[repeated] += [Tl]
                    if repeated not in a_repeated_mult:
                        a_repeated_mult[repeated] = [Tl]
                    else:
                        a_repeated_mult[repeated] += [Tl]
                    # right now rechecking independence
                    source = []
                    corresponding = []
                    for r in sorted(repeated_mult_L.keys()):
                        for point_lst in repeated_mult_L[r]:
                            for point in point_lst:
                                if isLinearlyIndependent(source + [point]):
                                    source.append(point)
                                    mult = point_to_mult_L[point]
                                    # if another point with this multiplier and level pair is in S
                                    # then the multiplier level pair will be the last element of corresponding
                                    if len(corresponding) != 0:
                                        if corresponding[-1][0] == mult:
                                            corresponding[-1][1] += 1
                                        else:
                                            corresponding.append([mult, 1])
                                    else:
                                        corresponding.append([mult, 1])
                                if len(source) == n+2:
                                    more = False
                                    break
                            if len(source) == n+2:
                                break
                        if len(source) == n+2:
                            break
                if not more:
                    break
            if not more:
                break
        if found_no_more:
            raise ValueError('no more rational preimages')
        i_repeated_mult = deepcopy(a_repeated_mult)
        a_repeated_mult = {}
        found_no_more = True
        #print('i_repeated_mult:', i_repeated_mult)
    #print('source:', source)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    #print('repeated_mult_L keys:', sorted(repeated_mult_L.keys()))
    possible_targets = []
    subset_iterators = []
    for tup in corresponding:
        possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
    #print('possible_targets:', possible_targets)
    for lst in possible_targets:
        subset_iterators.append(Subsets(range(len(lst[0])), lst[1]))
    #print('subset_iterators:', subset_iterators)
    #print('cardinalities:', [itr.cardinality() for itr in subset_iterators])
    count=0
    count2 = 0
    Conj = []
    for tup in product(*subset_iterators):
        target_set = []
        for i in range(len(tup)):
            for j in tup[i]:
                target_set.append(possible_targets[i][0][j])
        if isLinearlyIndependent(target_set):
            subset_arrangements = []
            for subset in tup:
                subset_arrangements.append(Arrangements(subset, len(subset)))
            for tup in product(*subset_arrangements):
                count += 1
                current_target = []
                for i in range(len(tup)):
                    for j in tup[i]:
                        current_target.append(possible_targets[i][0][j])
                phi = f.domain().point_transformation_matrix(current_target, source)
                if f.conjugate(phi) == g:
                    Conj.append(phi)
        else:
            count2 += 1
    #print('number of targets tried:', count)
    #print('number of not independent targets:', count2)
    return Conj

In [14]:
%%timeit
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x^3, y^3, z^3])
conj_set_preimage_no_independence(f, f)

66.1 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x^3, y^3, z^3])
conj_set_preimage_independece(f, f)

67.1 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x^3,x^2*y+y^3,z^3])
conj_set_preimage_no_independence(f, f)

32.6 ms ± 968 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%%timeit
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x^3,x^2*y+y^3,z^3])
conj_set_preimage_independece(f, f)

33.9 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%%timeit
K.<v>=CyclotomicField(3)
P.<x,y,z> = ProjectiveSpace(K, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
conj_set_preimage_no_independence(f, f)

14 s ± 304 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
K.<v>=CyclotomicField(3)
P.<x,y,z> = ProjectiveSpace(K, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
conj_set_preimage_independece(f, f)

13.4 s ± 150 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
P.<x,y,z,w,j,t,h> = ProjectiveSpace(QQ, 6)
f = DynamicalSystem([x^2, y^2, z^2, w^2, j^2, t^2, h^2])
conj_set_preimage_no_independence(f, f)

23.6 s ± 195 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
P.<x,y,z,w,j,t,h> = ProjectiveSpace(QQ, 6)
f = DynamicalSystem([x^2, y^2, z^2, w^2, j^2, t^2, h^2])
conj_set_preimage_independece(f, f)

23.1 s ± 278 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
